Устанавливаем пакет для непараметрической статистики

In [32]:
pip install PyNonpar


Note: you may need to restart the kernel to use updated packages.


Теперь его можно импортировать, как общепринятые пакеты

In [33]:
import PyNonpar
from PyNonpar import*



Применяем точного Вилкоксона (затабулированного)

In [34]:
x = [8,4,10,4,9,1,3,3,4,8]
y = [10,5,11,6,11,2,4,5,5,10]

PyNonpar.twosample.wilcoxon_mann_whitney_test(x, y, alternative="less", method = "exact", alpha = 0.05)

WilcoxonMannWhitneyResult(alternative='less', statistic=123.5, HodgesLehmann=1.5, lowerCI=-2, upperCI=5, pvalue=0.0837915954015025)

Теперь асимптотического

In [35]:
PyNonpar.twosample.wilcoxon_mann_whitney_test(x, y, alternative="less", method = "asymptotic", alpha = 0.05)

WilcoxonMannWhitneyResult(alternative='less', statistic=1.4096415789863137, HodgesLehmann=1.5, lowerCI=-2, upperCI=5, pvalue=0.07932277156815992)

HodgesLehmann - это оценка сдвига - медиана Ходжеса-Лемана - можно прочитать про нее в Холлендере-Вульфе

Теперь попробуем с файлом

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats


Поскольку файл дикий  - придется читать его с помощью pandas

In [37]:
data = pd.read_csv("covid_short_data-5.csv")
data.head()

,reporting date,location,country,gender,age,symptom_onset,exposure_start,exposure_end,visiting Wuhan,from Wuhan,death,recovered
0,1/20/2020,"Shenzhen, Guangdong",China,male,66.0,01/03/20,12/29/2019,01/04/20,1,0.0,0,0
1,1/20/2020,Shanghai,China,female,56.0,1/15/2020,NaN,01/12/20,0,1.0,0,0
2,1/21/2020,Zhejiang,China,male,46.0,01/04/20,NaN,01/03/20,0,1.0,0,0
3,1/21/2020,Tianjin,China,female,60.0,NaN,NaN,NaN,1,0.0,0,0
4,1/21/2020,Tianjin,China,male,58.0,NaN,NaN,NaN,0,0.0,0,0


In [38]:
type(data)

pandas.core.frame.DataFrame

In [39]:
print(data['age'][0:10])

0    66.0
1    56.0
2    46.0
3    60.0
4    58.0
5    44.0
6    34.0
7    37.0
8    39.0
9    56.0
Name: age, dtype: float64


symptom_onset - дата появления симптомов

exposure_start - дата начала контакта с вирусом

exposure_end - дата окончания контакта с вирусом


Посчитаем бессмысленную характеристику, чтобы не тратить зря осмысленные. Посчитаем есть ли различия в длине периода от заболевания до даты отчета у мужчин и женщин


Работа с датами

In [40]:
from datetime import datetime

In [41]:

def to_date(date):
    if len(date.split('/')[-1])==2:
        return datetime.strptime(date, '%m/%d/%y').date()
    return datetime.strptime(date, '%m/%d/%Y').date()
def numOfDays(date1, date2): 
    return (to_date(date2)-to_date(date1)).days 
      
date1 = '12/31/2019'
date2 = '02/12/20'
print(numOfDays(date1, date2), "days") 
type(numOfDays(date1, date2))

43 days


int

Применяем к файлу

In [42]:
daysMale = np.array([numOfDays(data['symptom_onset'][n], data['reporting date'][n]) 
                     for n in range(len(data['reporting date'])) 
                     if type(data['symptom_onset'][n]) == str 
                     and type(data['reporting date'][n]) == str 
                     and data['gender'][n] == 'male'])

In [43]:
print(daysMale)

[17 17 10  7 12  5  5  6 12 12 11  8  8  3  2  2  2  2  3  3  2  9  2  4
  4  4 33 23 14  9 12 10 13 31  7 17 20 16 17  6  6  5  0  3  3 17 25 14
  4  5  9 13 13 15 12 21 11 14 12 32  3  8  8  5  1  1  1  4 11  2  2  5
  4  4  5  7  7  4  3  7  1  9  5  1  5  0  1  7  9 10  4  5  2  6  3  3
 12 10  3  6 14  1  5  6 10 12 12  2  3 15 13 11 13 25 11 14  3 11 13  1
 10  0 14 10  2  2  9 11 13  5 16 11 16  8 15  7 14  1  6  6  3  5 15  3
  2  2  7 20 19  6 12 10  2  2  6  5  2 11  7  8 10  7 10 11  5  6  5  5
  4  4 14  9  3  2  4 14 12  2  1  2  3 13 11  7  7  2  2 10 11  9 12  6
  6  3  6  3  7  5  5  1  1  1  1 12  7  3 22  3  3  3  4  5  8  3  8 12
  4  9 16 15  9  9 10  9  8  8 10 10  6  5 11  9  5  9  6  3 14  1  2  3
  7  8 11 16  4 14  5  7  4  2 22  5  7 19 14  3  6  5  1  0  1 10  8 19
 20  1  2 18  3 11 12 20 20 26  3  3  3 11  5  4  2  2  9  6  3  2 10  1
  5  2  1  6  2  2  4 10  6  1  5  1  4  1  7  2  1  8  0 14 13  7  6  3
  4 10 10  2 11  9  6 12 11  8  9 15  5 12  9  5  7

In [44]:
daysFemale = np.array([numOfDays(data['symptom_onset'][n], data['reporting date'][n]) 
                       for n in range(len(data['reporting date'])) 
                       if type(data['symptom_onset'][n]) == str 
                       and type(data['reporting date'][n]) == str 
                       and data['gender'][n] == 'female'])

In [45]:
stats.shapiro(daysMale)

(0.9008049368858337, 3.8859787918977054e-14)

In [46]:
stats.shapiro(daysFemale)

(0.8628406524658203, 4.725056066898214e-13)

In [47]:
daysMalemini = np.array([numOfDays(data['symptom_onset'][n], data['reporting date'][n]) 
                     for n in range(30) 
                     if type(data['symptom_onset'][n]) == str 
                     and type(data['reporting date'][n]) == str 
                     and data['gender'][n] == 'male'])

In [48]:
daysFemalemini = np.array([numOfDays(data['symptom_onset'][n], data['reporting date'][n]) 
                       for n in range(50) 
                       if type(data['symptom_onset'][n]) == str 
                       and type(data['reporting date'][n]) == str 
                       and data['gender'][n] == 'female'])

In [49]:
stats.shapiro(daysMalemini)

(0.9288505911827087, 0.20820747315883636)

In [50]:
stats.shapiro(daysFemalemini)

(0.848726749420166, 0.07215212285518646)

In [51]:
print(len(daysMalemini))
print(len(daysFemalemini))

17
9


Ну вот вдруг у кого-то получится вот так - что нормальность не отвергается (p-value >0.05)

Тогда можно Стьюдента  (Ttest)

Только сперва нужно проверить равенство дисперсий

In [52]:
def Ftest_pvalue(d1,d2):
    df1 = len(d1) - 1
    df2 = len(d2) - 1
    F = d1.var(ddof=1)/d2.var(ddof=1)
    double_tailed_pval = 2*stats.f.cdf(F,df1,df2) if F<1 else 2*(1-stats.f.cdf(F,df1,df2))
    return double_tailed_pval

In [53]:
Ftest_pvalue(daysMalemini, daysFemalemini)

0.3909077267279053

Значит можно применять Т - тест с равными дисперсиями

In [54]:
stats.ttest_ind(daysMale, daysFemale)

Ttest_indResult(statistic=0.20323400501910402, pvalue=0.8390264901057278)

Есть минимум две реализации двухвыборочного Вилкоксона

Этот работает с массивами и годится только для достаточно больших выборок (n,m>30) 
потому что реализован только асимптотически

In [55]:
stats.ranksums(daysMale, daysFemale)

RanksumsResult(statistic=0.32774663784970953, pvalue=0.7431032373745743)




А этот работает со списками и есть в точном варианте - можно задать method = "exact" - это важно для тех, у кого получатся небольшие выборки

In [56]:
PyNonpar.twosample.wilcoxon_mann_whitney_test(list(daysMale), list(daysFemale), alternative="two.sided", method = "asymptotic", alpha = 0.05)

WilcoxonMannWhitneyResult(alternative='two.sided', statistic=-0.32852302329989425, HodgesLehmann=0.0, lowerCI=-1, upperCI=1, pvalue=0.7425162393218762)

In [57]:
PyNonpar.twosample.wilcoxon_mann_whitney_test(list(daysMalemini), list(daysFemalemini), alternative="two.sided", method = "exact", alpha = 0.05)

WilcoxonMannWhitneyResult(alternative='two.sided', statistic=100.5, HodgesLehmann=-2.0, lowerCI=-6, upperCI=1, pvalue=0.2697905298362964)

Ну) видим, что pvalue в одном случае громадный а в другом просто большой - то есть данные совсем не противоречат тому, что различий нет)

In [58]:
print(daysMale)
print(daysFemale)

[17 17 10  7 12  5  5  6 12 12 11  8  8  3  2  2  2  2  3  3  2  9  2  4
  4  4 33 23 14  9 12 10 13 31  7 17 20 16 17  6  6  5  0  3  3 17 25 14
  4  5  9 13 13 15 12 21 11 14 12 32  3  8  8  5  1  1  1  4 11  2  2  5
  4  4  5  7  7  4  3  7  1  9  5  1  5  0  1  7  9 10  4  5  2  6  3  3
 12 10  3  6 14  1  5  6 10 12 12  2  3 15 13 11 13 25 11 14  3 11 13  1
 10  0 14 10  2  2  9 11 13  5 16 11 16  8 15  7 14  1  6  6  3  5 15  3
  2  2  7 20 19  6 12 10  2  2  6  5  2 11  7  8 10  7 10 11  5  6  5  5
  4  4 14  9  3  2  4 14 12  2  1  2  3 13 11  7  7  2  2 10 11  9 12  6
  6  3  6  3  7  5  5  1  1  1  1 12  7  3 22  3  3  3  4  5  8  3  8 12
  4  9 16 15  9  9 10  9  8  8 10 10  6  5 11  9  5  9  6  3 14  1  2  3
  7  8 11 16  4 14  5  7  4  2 22  5  7 19 14  3  6  5  1  0  1 10  8 19
 20  1  2 18  3 11 12 20 20 26  3  3  3 11  5  4  2  2  9  6  3  2 10  1
  5  2  1  6  2  2  4 10  6  1  5  1  4  1  7  2  1  8  0 14 13  7  6  3
  4 10 10  2 11  9  6 12 11  8  9 15  5 12  9  5  7

Пошло моё решение:

In [103]:
ageSing = np.array([data['age'][n]
    for n in range(len(data['country']))
    if data['country'][n] == 'Singapore'
    and (data['gender'][n] == 'male'
    or data['gender'][n] == 'female')
    and n != 474
])

In [104]:
print(ageSing)

[66.  53.  37.  36.  56.  56.  35.  56.  56.  56.  31.  37.  73.  31.
 47.  38.  47.  31.  28.  48.  44.  32.  42.   0.5 45.  41.  27.  53.
 42.  39.  40.  64.  38.  53.  52.  51.  36.  71.  39.  54.  37.   2.
 35.  39.  34.  46.  62.  48.  37.  54.  55.  30.  30.  26.  55.  61.
 51.  57.  44.  54.  50.  61.  28.  56.  79.  26.  27.  25.  40.  43.
 29.  71.   1.  35.  57.  35.  38.  50.  57.  54.  35.  36.  24.  32.
 30.  41.  75.  58.  47.  38. ]


In [86]:
ageSouth = np.array([data['age'][n]
    for n in range(len(data['country']))
    if data['country'][n] == 'South Korea'
    and (data['gender'][n] == 'male'
    or data['gender'][n] == 'female')
])

In [87]:
print(ageSouth)

[35. 55. 54. 55. 32. 56. 28. 62. 28. 52. 25. 48. 28. 40. 43. 42. 37. 21.
 36. 41. 59. 46. 58. 28. 73. 51. 37. 30. 82. 68. 61. 11. 40. 24. 26. 48.
 49. 57. 61. 77. 69. 29. 58. 46. 54. 28. 63. 72. 58. 76. 61. 24. 38. 57.
 59. 75. 30. 26. 34. 59. 58. 60. 52. 59. 50. 22. 30. 56. 29. 48. 56. 39.
 20. 49. 23. 36. 57. 55. 22. 34. 65. 64. 67. 74. 35. 62. 58. 62. 41. 36.
 69. 75.]


In [109]:
stats.shapiro(ageSing)

(0.9672545194625854, 0.022729042917490005)

pvalue < 0.05, поэтому, применяя критерий Шапиро-Уилка, делаем вывод о том, что возраст в Сингапуре распределён не нормально

In [93]:
stats.shapiro(ageSouth)

(0.9679070711135864, 0.022825533524155617)

Аналогично, pvalue < 0.05, поэтому, применяя критерий Шапиро-Уилка, делаем вывод о том, что возраст в Южной Корее распределён не нормально

In [108]:
print(len(ageSouth))
print(len(ageSing))

92
90
